<a href="https://colab.research.google.com/github/zecakpm/maps/blob/main/GEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Random Forest for Land Cover Classification - Google Earth Engine

In [1]:
#Import libraries
import ee
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=fmA2T-X9yaI0bDHFgH1GcXEbOm_0QjUX-RNG0Z3JEfQ&tc=7j0I13WSG8SzJYhZTpCHPyg151h2ULnsEIEVOcpDcxk&cc=d9szpRo6ZtJBS6vzPNdkI6Tw6mNQ_P--e9GkPfG42wM

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk7rWNwXIFwT7RC9kWnvBIhd2AjMeJaedoUbwyRFxTcOWmTIEs4JBqU

Successfully saved authorization token.


In [19]:
#Import libraries
import folium
from folium.map import Layer

In [3]:
#Initiate Goole Earth Engine
ee.Initialize()

In [78]:
#ROI
roi = ee.Geometry.Point([-6.266155, 53.350140])
location = roi.centroid().coordinates().getInfo()[::-1]

#Load Landsat 8 data
bands = ['B1','B2','B3','B4','B5','B7']
image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
.filterDate('2021-01-01','2021-12-31') \
.filterBounds(roi) \
.sort('CLOUD_COVER') \
.first()

In [79]:
#Import ESRI data
lc = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m").mosaic().clip(image.geometry())
label = 'b1'

In [80]:
#Sample the input imagery to get featureCollection of training data
sample = image.addBands(lc).sample(**{
    'region':image.geometry(),
    'numPixels': 1000,
    'seed': 0
})

In [81]:
#Spliting data
sample = sample.randomColumn()
trainingSample = sample.filter('random <= 0.8')
validationSample = sample.filter('random > 0.8')

In [82]:
#Classification Model 
trainedClassifier = ee.Classifier.smileRandomForest(10).train(**{
    'features': trainingSample,
    'classProperty': label,
    'inputProperties': bands
})

model = image.classify(trainedClassifier)

In [84]:
# Define a method for displaying Earth Engine image tiles on a folium map
# Func from google documentation >> https://colab.research.google.com/github/giswqs/qgis-earthengine-examples/blob/master/Folium/ee-api-folium-setup.ipynb
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

In [85]:
#Map basic configuration / zoom
Map = folium.Map(location=location,zoom_start = 8)

In [86]:
#Defining the base map
basemaps = {'Google Satellite Hybrid': folium.TileLayer(
    tiles = 'https://mt1.google.com/vt/lyrs-s&x-{x}&y-{y}&z-{z}',
    attr = 'Google',
    name = 'Google Satellite Hybrid',
    overlay= True,
    control = True
)}

In [87]:
basemaps['Google Satellite Hybrid'].add_to(Map)

In [88]:
# Color dictionary
dict = { 
    "names": [
        "Water", # deep sky blue
        "Trees", # forest green
        "Grass", # pistachio
        "Flooded Vegetation", # mint green
        "Crops", #sunshine yellow
        "Scrub/Shrub", # pale tan
        "Built Area", # cherry red
        "Bare Ground", # off-white
        "Snow/Ice", # baby blue
        "Clouds" # silver
    ],

    "colors":[
      "#1A5BAB",
      "#358221",
      "#A7D282",
      "#87D19E",
      "#FFDB5C",
      "#EECFA8",
      "#ED022A",
      "#EDE9E4",
      "#F2FAFF",
      "#C8C8C8"
    ]
}

In [89]:
# Add map layers
Map.add_ee_layer(model,{'min':1,'max':10,'palette':dict['colors']},'Classified 2021')
visParamsTrue = {'bands': ['B4','B3','B2'], 'min': 0, 'max': 3000, 'gamma':1.4}
Map.add_ee_layer(image, visParamsTrue, 'Landsat 2021')

In [90]:
# Add map control
Map.add_child(folium.LayerControl())